In [1]:
# from transformers import DetrImageProcessor, DetrForObjectDetection
# import torch
# from PIL import Image
# import requests

# url = "/kaggle/input/coco-2017-dataset/coco2017/train2017/000000461973.jpg"
# image = Image.open(url)

# # you can specify the revision tag if you don't want the timm dependency
# processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
# model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

# inputs = processor(images=image, return_tensors="pt")
# outputs = model(**inputs)

# # convert outputs (bounding boxes and class logits) to COCO API
# # let's only keep detections with score > 0.9
# target_sizes = torch.tensor([image.size[::-1]])
# results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

# for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
#     box = [round(i, 2) for i in box.tolist()]
#     print(
#             f"Detected {model.config.id2label[label.item()]} with confidence "
#             f"{round(score.item(), 3)} at location {box}"
#     )


In [2]:
# from PIL import ImageDraw, ImageFont

# # Open the image again to draw on it
# image_with_boxes = image.copy()
# draw = ImageDraw.Draw(image_with_boxes)
# font = ImageFont.load_default()

# # Loop through the results and draw the boxes
# for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
#     box = [round(i, 2) for i in box.tolist()]
#     # Draw the rectangle
#     draw.rectangle(box, outline="red", width=2)
#     # Draw the label and score
#     label_text = f"{model.config.id2label[label.item()]}: {round(score.item(), 3)}"
#     text_bbox = font.getbbox(label_text)
#     text_width = text_bbox[2] - text_bbox[0]
#     text_height = text_bbox[3] - text_bbox[1]
#     draw.rectangle([box[0], box[1], box[0] + text_width, box[1] + text_height], fill="red")
#     draw.text((box[0], box[1]), label_text, fill="white", font=font)

# # Save the image with bounding boxes to a file
# output_path = "/tmp/output_with_boxes.png"
# image_with_boxes.save(output_path)
# print(f"Image saved to {output_path}")

In [3]:
# from IPython.display import Image as IPImage, display

# display(IPImage(filename=output_path))

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import os
import yaml
import json

In [5]:
!git clone https://github.com/facebookresearch/detr.git
!pip install pycocotools

Cloning into 'detr'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 265 (delta 0), reused 1 (delta 0), pack-reused 264
Receiving objects: 100% (265/265), 12.88 MiB | 32.40 MiB/s, done.
Resolving deltas: 100% (123/123), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 12.8 MB/s eta 0:00:00


In [6]:
import os
import shutil
import json

# Define source paths
source_data_path = '/kaggle/input/coco-2017-dataset/coco2017/train2017'
source_train_annotations = '/kaggle/input/semis-od-coco-10/instances_train2017_labeled.json'
source_val_annotations = '/kaggle/input/semis-od-coco-10/instances_val2017.json'

# Define destination paths
destination_path = 'coco'
annotations_path = os.path.join(destination_path, 'annotations')
train_images_path = os.path.join(destination_path, 'train2017')
val_images_path = os.path.join(destination_path, 'val2017')

# Create the directory structure
os.makedirs(annotations_path, exist_ok=True)
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(val_images_path, exist_ok=True)

# Copy annotation files
shutil.copy(source_train_annotations, os.path.join(annotations_path, 'instances_train2017.json'))
shutil.copy(source_val_annotations, os.path.join(annotations_path, 'instances_val2017.json'))

# Function to get image filenames from annotations
def get_image_filenames(annotation_file):
    with open(annotation_file, 'r') as f:
        data = json.load(f)
    image_filenames = {image['file_name']: image['id'] for image in data['images']}
    return image_filenames

# Get image filenames for train and val sets
train_images = get_image_filenames(source_train_annotations)
val_images = get_image_filenames(source_val_annotations)

# Copy images to respective directories based on annotation files
for image_filename in os.listdir(source_data_path):
    if image_filename in train_images:
        shutil.copy(os.path.join(source_data_path, image_filename), os.path.join(train_images_path, image_filename))
    elif image_filename in val_images:
        shutil.copy(os.path.join(source_data_path, image_filename), os.path.join(val_images_path, image_filename))

print("Dataset organized successfully.")

Dataset organized successfully.


In [7]:
!torchrun \
    --nproc_per_node 2 \
    /kaggle/working/detr/main.py \
    --num_workers 2 \
    --batch_size 4 \
    --device 'cuda' \
    --coco_path /kaggle/working/coco \
    --epochs 15 \
    --output_dir /kaggle/working/weight \
    --resume /kaggle/input/dert-pretrained-weight/detr-r50-e632da11.pth
#     --resume https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth \

[2024-06-01 13:33:15,975] torch.distributed.run: [WARNING] 
[2024-06-01 13:33:15,975] torch.distributed.run: [WARNING] *****************************************
[2024-06-01 13:33:15,975] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-06-01 13:33:15,975] torch.distributed.run: [WARNING] *****************************************
| distributed init (rank 0): env://
| distributed init (rank 1): env://
git:
  sha: 29901c51d7fe8712168b8d0d64351170bc0f83e0, status: clean, branch: main

Namespace(lr=0.0001, lr_backbone=1e-05, batch_size=4, weight_decay=0.0001, epochs=15, lr_drop=200, clip_max_norm=0.1, frozen_weights=None, backbone='resnet50', dilation=False, position_embedding='sine', enc_layers=6, dec_layers=6, dim_feedforward=2048, hidden_dim=256, dropout=0.1, nheads=8, num_queries=100, pre_n

In [8]:
# !python /kaggle/working/detr/main.py \
#     --batch_size 4 \
# #     --no_aux_loss \
# #     --eval \
#     --resume https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth \
#     --coco_path /kaggle/working/coco